In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis.dihedrals import Dihedral, Ramachandran
from math import floor
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
sns.set_style("ticks", {"xtick.direction": "in","ytick.direction": "in", "xtick.major.size": 8, "ytick.major.size": 8, 'font.family': ['FreeSans']})
import numpy as np
from pickle import dump, load

In [2]:
Nbins = 24

In [22]:
# XTC = '../ensemble/ensemble.dcd'
# PDB = '../ensemble/18.rebuilt.pdb'
# u = mda.Universe(PDB, XTC)
# sim='../ensemble'
# step=1#int(floor(len(u.trajectory)/1000))

# resnums = [el.resnum for el in u.residues][1:-1]

# #calculate all rama
# R = dict()
# mats = dict()

# for el in resnums:
#     r = u.select_atoms("resid "+str(el))
#     R[el] = Ramachandran(r).run(step=step)
    
#     phi = R[el].angles[:, :, 0].flatten()
#     psi = R[el].angles[:, :, 1].flatten()
    
#     temp = np.histogram2d(psi, phi, bins=Nbins, range=[[-180,180], [-180,180]], density=True)[0]
#     mats[el] = temp/np.sum(el)
    
# dump(R, open( sim+"/R.pkl", "wb" ) )
# dump(mats, open( sim+"/mats.pkl", "wb" ) )

In [3]:
for eps in ["3e-5"]:
    for sigma in ["mixed"]:
        
        sim=eps+"_"+sigma
        
        XTC = sim+'/all.dcd'
        PDB = sim+'/1/CG.pdb'
        
        u = mda.Universe(PDB, XTC)
        step=int(floor(len(u.trajectory)/500))
        
        resnums = [el.resnum for el in u.residues][1:-1]

        #calculate all rama
        R = dict()
        mats = dict()
        
        for el in resnums:
            r = u.select_atoms("resid "+str(el))
            R[el] = Ramachandran(r).run(step=step)
    
            phi = R[el].angles[:, :, 0].flatten()
            psi = R[el].angles[:, :, 1].flatten()
    
            temp = np.histogram2d(psi, phi, bins=Nbins, range=[[-180,180], [-180,180]], density=True)[0]
            mats[el] = temp/np.sum(el)
    
        dump(R, open( sim+"/R.pkl", "wb" ) )
        dump(mats, open( sim+"/mats.pkl", "wb" ) )

/home/nsalvi/anaconda3/envs/py36/lib/python3.6/site-packages/MDAnalysis/topology/PDBParser.py:330: UserWarning: Element information is absent or missing for a few atoms. Elements attributes will not be populated.
  warnings.warn("Element information is absent or missing for a few "


In [4]:
# read ensemble results 
infile = open('../ensemble/mats.pkl','rb')
mats_ens = load(infile)
infile.close()

In [5]:
results = dict()

for eps in ["1e-4"]:
    results[eps] = dict()
    for sigma in ["0.25", "0.5", "0.75"]:
        sim=eps+"_"+sigma
        value = 0
        infile = open(sim+"/mats.pkl",'rb')
        mats_sim = load(infile)
        infile.close()
        for k in mats_ens.keys():
            ref = mats_ens[k]/np.sum(mats_ens[k])
            calc = mats_sim[k]/np.sum(mats_sim[k])
            value += np.sum((ref-calc)**2)
        results[eps][sigma] = value

In [6]:
results

{'1e-4': {'0.25': 5.49568921777204,
  '0.5': 3.585821886566187,
  '0.75': 2.378931953856757}}

In [7]:
results = dict()

for eps in ["3e-5"]:
    results[eps] = dict()
    for sigma in ["0.5", "0.625", "mixed"]:
        sim=eps+"_"+sigma
        value = 0
        infile = open(sim+"/mats.pkl",'rb')
        mats_sim = load(infile)
        infile.close()
        for k in mats_ens.keys():
            ref = mats_ens[k]/np.sum(mats_ens[k])
            calc = mats_sim[k]/np.sum(mats_sim[k])
            value += np.sum((ref-calc)**2)
        results[eps][sigma] = value

In [8]:
results

{'3e-5': {'0.5': 5.690280315018665,
  '0.625': 4.261044614762488,
  'mixed': 5.196233558232836}}